In [1]:
import qml
import numpy as np

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qi

# Import data

In [2]:
paths=qi.wrapper_alch_data()

In [3]:
# paths

In [4]:
# load data into list, count number of atoms per molecule
alchemy_data, molecule_size = qi.load_alchemy_data(paths)
max_size = np.amax(molecule_size)

# Generate Representation

In [5]:
full_matrix = qi.generate_atomic_representations(alchemy_data, molecule_size)

In [6]:
full_matrix.shape


(7584, 210)

# Calculate distance between representations

In [7]:
dist = qi.calculate_distances(full_matrix)

# Generate Label vector

In [8]:
energies = qi.generate_label_vector(alchemy_data, molecule_size.sum())

# Difference between labels

In [9]:
diff_shape = int(len(full_matrix)*(len(full_matrix)+1)/2)
start=0
width = len(full_matrix)
en_diff = np.empty(diff_shape)
for idx in range(0, len(full_matrix)):
    en_diff[start:start+width] = np.abs(energies[idx]-energies[idx:])
    start = start+width
    width -= 1

# Plots

In [10]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})


In [30]:
# select random indices
indices = np.random.choice(len(dist), 100000)
indices = np.sort(indices)

fig, ax = plt.subplots(1,1)

ax.plot(np.take(dist, indices), np.take(en_diff, indices), 'o')
ax.set_xlabel(r'$d_{ij} $')
ax.set_ylabel('$|e_i - e_j|$')

Text(0, 0.5, '$|e_i - e_j|$')

# Generate Kernel

In [13]:
import qml.kernels

In [25]:
full_matrix

array([[36.8581052 ,  2.90564209,  0.5       , ...,  0.        ,
         0.        ,  0.        ],
       [36.8581052 , 17.73882949, 73.51669472, ...,  0.        ,
         0.        ,  0.        ],
       [36.8581052 ,  2.90563574,  0.5       , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [53.3587074 , 19.54993299, 53.3587074 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5       ,  2.92143402, 36.8581052 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5       ,  2.92677017, 36.8581052 , ...,  0.        ,
         0.        ,  0.        ]])

In [28]:
gaussian_kernel = qml.kernels.gaussian_kernel(full_matrix, full_matrix, 100)

In [48]:
gaussian_kernel.shape

(7584, 7584)

In [ ]:
# test if kernels.gaussian_kernel does what we want by comparing the kernel elements to manually computed elements

In [36]:
def kernel_element(rep_matrix, i, j, sigma):
    d = np.linalg.norm(rep_matrix[i]-rep_matrix[j])
    element = np.exp(-np.power(d,2)/(2*sigma**2))
    return(element)

In [46]:
kernel_element(full_matrix, 0, 5, 100)

0.5159083702796291

In [47]:
gaussian_kernel[0,5]

0.515908370279629

# Calculate Regression Coefficients